In [1]:
import random
import mne, os, glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import random
from mne.datasets import somato
from mne.time_frequency import tfr_morlet
%matplotlib qt 

In [2]:
""" 
Load data
"""
# Big list
ep_list = []
# Sup grouped list
ep_group1 = []
ep_group2 = []
ep_group3 = []
ep_group4 = []
subjects = []
# Specify the epoch length we will be looking at
epoch_tmin = -0.1
epoch_tmax = 0.5

for sub in range(1,22):

    if sub == 20:
        continue
    cleaned_data_dir = '/Users/mvmigem/Documents/data/project_1/preprocessed/mastoid_ref/'
    clean_epoch_path = os.path.join(cleaned_data_dir,f'main_eventset_mastoidref_{sub:02}-epo.fif')
    epochs = mne.read_epochs(clean_epoch_path)
    epochs.info['bads']= []
    ep_list.append(epochs)

    # if epochs.metadata['loc_quad'].iloc[0] == 0:
    #     ep_group1.append(epochs)
    # elif epochs.metadata['loc_quad'].iloc[0] == 1:
    #     ep_group2.append(epochs)
    # elif epochs.metadata['loc_quad'].iloc[0] == 2:
    #     ep_group3.append(epochs)
    # elif epochs.metadata['loc_quad'].iloc[0] == 3:
    #     ep_group4.append(epochs)
    print(epochs.info['nchan'])

# Concat list into big epoch object
eps = mne.concatenate_epochs(ep_list).crop(tmin=epoch_tmin,tmax=epoch_tmax).apply_baseline()

Reading c:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_01-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 33 columns
4189 matching events found
No baseline correction applied
0 projection items activated
69
Reading c:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_02-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 33 columns
4196 matching events found
No baseline correction applied
0 projection items activated
69
Reading c:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_03-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 32 columns
4188 matching events f

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_22316\2384206660.py:37: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  eps = mne.concatenate_epochs(ep_list).crop(tmin=epoch_tmin,tmax=epoch_tmax).apply_baseline()


Adding metadata with 35 columns
82046 matching events found
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)


In [3]:

sub_list = eps.metadata['participant'].unique() 

# Divide the epoch file into sections based on metadata that can't be distinguished by event names
# Drop catch trials
epochs_nocatch = eps.copy()['catch_trial == 0']
# # Divide attention conditions
# epochs_attended = epochs_nocatch['attention == "attended"']
# epochs_unattended = epochs_nocatch['attention == "unattended"']
# # Divide by staring position
# epochs_start3 = epochs_nocatch['start_position == 2']
# epochs_start1 = epochs_nocatch['start_position == 0']
# epochs_start2 = epochs_nocatch['start_position == 1']
# epochs_start4 = epochs_nocatch['start_position == 3']
# # Divide unattended trials by start pos
# epochs_unattended_start3 = epochs_unattended['start_position == 2']
# epochs_unattended_start1 = epochs_unattended['start_position == 0']
# epochs_unattended_start2 = epochs_unattended['start_position == 1']
# epochs_unattended_start4 = epochs_unattended['start_position == 3']
# # Divide attented trials by start pos
# epochs_attended_start3 = epochs_attended['start_position == 2']
# epochs_attended_start1 = epochs_attended['start_position == 0']
# epochs_attended_start2 = epochs_attended['start_position == 1']
# epochs_attended_start4 = epochs_attended['start_position == 3']
# # Extra devision for the odd boys
# epochs_attended_reg = epochs_attended['expected =="regular"']
# epochs_attended_odd = epochs_attended['expected == "odd"']
# epochs_unattended_reg = epochs_unattended['expected =="regular"']
# epochs_unattended_odd = epochs_unattended['expected == "odd"']

### Evoked plotting

In [ ]:
"""
Main pos individual channels
"""
evoked_pos1_list = []
evoked_pos2_list = []
evoked_pos3_list = []
evoked_pos4_list = []
for i,sub in enumerate(sub_list):
    evoked_pos1_list.append(epochs_nocatch['pos1'][f'participant == {sub}'].average())
    evoked_pos2_list.append(epochs_nocatch['pos2'][f'participant == {sub}'].average())
    evoked_pos3_list.append(epochs_nocatch['pos3'][f'participant == {sub}'].average())
    evoked_pos4_list.append(epochs_nocatch['pos4'][f'participant == {sub}'].average())

evoked_pos1 = mne.grand_average(evoked_pos1_list)
evoked_pos2 = mne.grand_average(evoked_pos2_list)
evoked_pos3 = mne.grand_average(evoked_pos3_list)
evoked_pos4 = mne.grand_average(evoked_pos4_list)

evokeds_list = [evoked_pos1,evoked_pos2,evoked_pos3,evoked_pos4]
conds = ('pos1','pos2','pos3','pos4')
# conds = ('seq1','seq2','seq3','seq4')

norm = dict(zip(conds, evokeds_list))

In [4]:
# Event selection for prediction effects

# Remove catch
# epochs_nocatch = eps.copy()['catch_trial == 0']['pos4']
# Select second stim of trial
epochs_stim2 = epochs_nocatch['seq2']
# Regular vs odd
epochs_odd = epochs_stim2['expected == "odd"']
epochs_reg = epochs_stim2['expected == "regular"']['precedes_odd == 1']
# Attention
ep_odd_att = epochs_odd['attention == "attended"']
ep_odd_unatt = epochs_odd['attention == "unattended"']
ep_reg_att = epochs_reg['attention == "attended"']
ep_reg_unatt = epochs_reg['attention == "unattended"']


In [ ]:
big_meta = epochs_nocatch['seq2'].metadata

In [ ]:
"""
checking prediction manipulation on p3, means agragating across positions 
"""
att_reg_ev = []
att_odd_ev = []
unatt_reg_ev = []
unatt_odd_ev = []
# totals_grand = []


for i, sub in enumerate(sub_list):
    
    att_reg_ev.append(
        epochs_nocatch['seq2'][f"participant == {sub}"]["attention == 'attended'"]['expected == "regular"']['precedes_odd == 1'].average())
    att_odd_ev.append(
        epochs_nocatch['seq2'][f"participant == {sub}"]["attention == 'attended'"]['expected == "odd"'].average())
    unatt_reg_ev.append(
        epochs_nocatch['seq2'][f"participant == {sub}"]["attention == 'unattended'"]['expected == "regular"']['precedes_odd == 1'].average())
    unatt_odd_ev.append(
        epochs_nocatch['seq2'][f"participant == {sub}"]["attention == 'unattended'"]['expected == "odd"'].average())
    
    # totals_grand.append(
    #     epochs_nocatch['seq2'].average())
    

In [ ]:

evoked_pred = [mne.grand_average(att_reg_ev),
               mne.grand_average(att_odd_ev),
               mne.grand_average(unatt_reg_ev),
               mne.grand_average(unatt_odd_ev),]

conditions = ('attended regular','attended odd','unattended regular','unattended odd')

norm = dict(zip(conditions, evoked_pred))

In [ ]:
""" 
create df with the neededed values for p3 analysis
"""
# init df
long_sub_list = [item for item in sub_list for _ in range(4)]
p3_df = pd.DataFrame(long_sub_list, columns=['subject'])

# selected channels
roi = ['P1','Pz','P2','CP1','CPz','CP2','C1','Cz','C2']
roi_ix = mne.pick_channels(epochs_nocatch[0].info["ch_names"], include=roi)
roi_dict = dict(centropar = list(roi_ix))
p3_tmin, p3_tmax = 0.25, 0.45

# loop subjects
mid_att = []
mid_pred = []
mid_peak = []
mid_lat = []
mid_mean = []
sub_check = []
for i, sub in enumerate(sub_list):
    print(sub)
    # attended and regular
    ev_roi = mne.channels.combine_channels(inst = att_reg_ev[i], groups= roi_dict)
    mean_amp = ev_roi.crop(tmin=p3_tmin,tmax=p3_tmax).data.mean(axis=1) * 1e6
    mid_att.append('attended')
    mid_pred.append('regular')
    mid_mean.append(mean_amp[0])
    sub_check.append(sub)
    # attended and odd
    ev_roi = mne.channels.combine_channels(inst = att_odd_ev[i], groups= roi_dict)
    mean_amp = ev_roi.crop(tmin=p3_tmin,tmax=p3_tmax).data.mean(axis=1) * 1e6
    mid_att.append('attended')
    mid_pred.append('odd')
    mid_mean.append(mean_amp[0])
    sub_check.append(sub)
    # unattended and regular
    ev_roi = mne.channels.combine_channels(inst = unatt_reg_ev[i], groups= roi_dict)
    mean_amp = ev_roi.crop(tmin=p3_tmin,tmax=p3_tmax).data.mean(axis=1) * 1e6
    mid_att.append('unattended')
    mid_pred.append('regular')
    mid_mean.append(mean_amp[0])
    sub_check.append(sub)
    # unattended and odd
    ev_roi = mne.channels.combine_channels(inst = unatt_odd_ev[i], groups= roi_dict)
    mean_amp = ev_roi.crop(tmin=p3_tmin,tmax=p3_tmax).data.mean(axis=1) * 1e6
    mid_att.append('unattended')
    mid_pred.append('odd')
    mid_mean.append(mean_amp[0])
    sub_check.append(sub)

p3_df['attention'] = mid_att
p3_df['expectation'] = mid_pred
p3_df['mean_amp'] = mid_mean



In [ ]:
p3_df.to_csv(r'C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes\p3_df.csv',index= False)

In [5]:
"""
Main effects 
"""
reg_att_up = []
odd_att_up = []
reg_unatt_up = []
odd_unatt_up = []

reg_att_down = []
odd_att_down = []
reg_unatt_down = []
odd_unatt_down = []

cond_ep_lists = [ep_reg_att,ep_odd_att,ep_reg_unatt,ep_odd_unatt] 
up_conds = [reg_att_up,odd_att_up,reg_unatt_up,odd_unatt_up]  
down_conds = [reg_att_down,odd_att_down,reg_unatt_down,odd_unatt_down]  
ev_names = ['reg_att','odd_att','reg_unatt','odd_unatt']
# List to know which position was used to select the up or down later
sub_pos_info = []

evokeds_up = dict(zip(ev_names,up_conds))
evokeds_down = dict(zip(ev_names,down_conds))

for i, sub in enumerate(sub_list):
    # which set of positions does this sub have
    loc_quad_values = ep_reg_att[f'participant == {sub}'].metadata["loc_quad"]
    loc_quad_value = loc_quad_values.unique()[0]
    # Dict to stor positional info
    sub_info_dict = {}
    sub_info_dict['subject'] = sub
    # Positions 1 and 3
    if loc_quad_value == 0  or loc_quad_value== 2 :
        # make dict to store the selected evokeds
        ev_position1 = {}
        # for every condition
        for i, epoch in enumerate(cond_ep_lists):
            # transform the epochs into evokeds
            ev_position1[ev_names[i]] = epoch['pos1'][f'participant == {sub}'].average()
        # same for position 3
        ev_position3 = {}
        for i, epoch in enumerate(cond_ep_lists):
            ev_position3[ev_names[i]] = epoch['pos3'][f'participant == {sub}'].average()
        # for every condition in the upper vf
        for cond,evoked in ev_position1.items():
            evokeds_up[cond].append(evoked)
        # same for down
        for cond,evoked in ev_position3.items():
            evokeds_down[cond].append(evoked)
        # Store position information
        sub_info_dict['up_pos'] = 1
        sub_info_dict['down_pos'] = 3
    # same for positions 2 and 4
    if loc_quad_value == 1  or loc_quad_value== 3 :
        ev_position2 = {}
        for i, evoked in enumerate(cond_ep_lists):
            ev_position2[ev_names[i]] = evoked['pos2'][f'participant == {sub}'].average()
        ev_position4 = {}
        for i, evoked in enumerate(cond_ep_lists):
            ev_position4[ev_names[i]] = evoked['pos4'][f'participant == {sub}'].average()
        # for every condition in the upper vf
        for cond,evoked in ev_position2.items():
            # append to that conditions list in the up_dict
            evokeds_up[cond].append(evoked)
        # same for lower vf
        for cond,evoked in ev_position4.items():
            evokeds_down[cond].append(evoked)
        sub_info_dict['up_pos'] = 2
        sub_info_dict['down_pos'] = 4
    # Add info_dict_to list
    sub_pos_info.append(sub_info_dict)



NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

In [6]:
# Load peak properties of localiser data
peak_properties = pd.read_csv(r'C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes\c1_peak_properties.csv')

In [7]:
# transfrom dict of lists to list of dicts for plotting individually
subj_uplist = [dict(zip(evokeds_up, values)) for values in zip(*evokeds_up.values())]
subj_downlist = [dict(zip(evokeds_down, values)) for values in zip(*evokeds_down.values())]
scale = [-12,12]
# for i in range(20):
#     the_row = peak_properties[peak_properties['subject'] == sub_pos_info[i]['subject']]
#     pos = sub_pos_info[i]['down_pos']
#     channel = the_row[f'pos{pos}_peak_channel'].iloc[0]
#     mne.viz.plot_compare_evokeds(subj_downlist[i], picks= channel, vlines=[0.05,0.1],ylim=dict(eeg=scale))

In [8]:
# Create dataframes for export and analysis
long_sub_list = [item for item in sub_list for _ in range(4)]
c1_long_df = pd.DataFrame(long_sub_list, columns=['subject'])
window_halfwidth = 0.01
# Condition columns
mid_att = []
mid_pred = []
# General analysis list init 
genup_amp = []
gendown_amp = []
general_amp = []
# General time window estimate
gen_lat = peak_properties['grand_average_peak_latency'][0]
gen_tmin = gen_lat - window_halfwidth
gen_tmax = gen_lat + window_halfwidth
# 
selectup_gen_evoked = []
selectdown_gen_evoked = []

# Tailored analysis empty lists
tailup_amp = []
taildown_amp = []
tailored_amp =[]

selectup_tail_evoked = []
selectdown_tail_evoked = []

# Ultra tailored analysis values 
ultraup_amp = []
ultradown_amp = []
ultra_amp = []

selectup_ultra_evoked = []
selectdown_ultra_evoked = []

conditions_att = ['attended','attended','unattended','unattended',]
conditions_pred = ['regular','odd','regular','odd',]
conditions = ['reg_att','odd_att','reg_unatt','odd_unatt']

# Static C1 window analysis
for i, sub in enumerate(sub_list):
    # Dicts to store the selected evoked
    upgen_select_dict = {}
    downgen_select_dict = {}
    uptail_select_dict = {}
    downtail_select_dict = {}
    upultra_select_dict = {}
    downultra_select_dict = {}
    # Select the channel and latency based on localiser info
    sub_row = peak_properties[peak_properties['subject'] == sub_pos_info[i]['subject']]
    # Tailored info
    tailored_channel = sub_row['all_pos_peak_channel'].iloc[0]
    tailored_latency = sub_row['all_pos_peak_latency'].iloc[0]
    tail_tmin = tailored_latency - window_halfwidth
    tail_tmax = tailored_latency + window_halfwidth
    # Ultra tailored info
    up_pos = sub_pos_info[i]['up_pos']
    down_pos = sub_pos_info[i]['down_pos']
    ultraup_channel = sub_row[f'pos{up_pos}_peak_channel'].iloc[0]
    ultradown_channel = sub_row[f'pos{down_pos}_peak_channel'].iloc[0]
    ultraup_latency = sub_row[f'pos{up_pos}_peak_latency'].iloc[0]
    ultradown_latency = sub_row[f'pos{down_pos}_peak_latency'].iloc[0]
    ultraup_tmin = ultraup_latency - window_halfwidth
    ultraup_tmax = ultraup_latency + window_halfwidth
    ultradown_tmin = ultradown_latency - window_halfwidth
    ultradown_tmax = ultradown_latency + window_halfwidth

    for ind in range(4):
        # attended and regular
        mid_att.append(conditions_att[ind])
        mid_pred.append(conditions_pred[ind])
        # General
        # Pick channel
        gen_selected_up = subj_uplist[i][conditions[ind]].copy().pick(['POz']) 
        gen_selected_down = subj_downlist[i][conditions[ind]].copy().pick(['POz'])
        upgen_select_dict[conditions[ind]] =  gen_selected_up
        downgen_select_dict[conditions[ind]] =  gen_selected_down
        # Extract data
        gen_up_mean_data = gen_selected_up.copy().crop(tmin=gen_tmin,tmax=gen_tmax).data
        gen_down_mean_data = gen_selected_down.copy().crop(tmin=gen_tmin,tmax=gen_tmax).data
        # Agragate and append
        genup_mean = gen_up_mean_data.mean(axis=1) * 1e6
        gendown_mean = gen_down_mean_data.mean(axis=1) * 1e6
        genup_amp.append(genup_mean[0])
        gendown_amp.append(gendown_mean[0])
        # RMS
        gen_mean = np.mean([np.sqrt(genup_mean**2),np.sqrt(gendown_mean**2)])
        general_amp.append(gen_mean)

        # Tailored
        # Pick channel
        tail_selected_up = subj_uplist[i][conditions[ind]].copy().pick([tailored_channel]) 
        tail_selected_down = subj_downlist[i][conditions[ind]].copy().pick([tailored_channel])
        uptail_select_dict[conditions[ind]] =  tail_selected_up
        downtail_select_dict[conditions[ind]] =  tail_selected_down
        # Extract data
        tail_up_mean_data = tail_selected_up.copy().crop(tmin=tail_tmin,tmax=tail_tmax).data
        tail_down_mean_data = tail_selected_down.copy().crop(tmin=tail_tmin,tmax=tail_tmax).data
        # Agragate and append
        tailup_mean = tail_up_mean_data.mean(axis=1) * 1e6
        taildown_mean = tail_down_mean_data.mean(axis=1) * 1e6
        tailup_amp.append(tailup_mean[0])
        taildown_amp.append(taildown_mean[0])
        # RMS
        tail_mean = np.mean([np.sqrt(tailup_mean**2),np.sqrt(taildown_mean**2)])
        tailored_amp.append(tail_mean)

        # Ultra tailored
        # Pick channel
        ultra_selected_up = subj_uplist[i][conditions[ind]].copy().pick([ultraup_channel]) 
        ultra_selected_down = subj_downlist[i][conditions[ind]].copy().pick([ultradown_channel])
        upultra_select_dict[conditions[ind]] =  ultra_selected_up
        downultra_select_dict[conditions[ind]] =  ultra_selected_down
        # Extract data
        ultra_up_mean_data = ultra_selected_up.copy().crop(tmin=ultraup_tmin,tmax=ultraup_tmax).data
        ultra_down_mean_data = ultra_selected_down.copy().crop(tmin=ultradown_tmin,tmax=ultradown_tmax).data
        # Agragate and append
        ultraup_mean = ultra_up_mean_data.mean(axis=1) * 1e6
        ultradown_mean = ultra_down_mean_data.mean(axis=1) * 1e6
        ultraup_amp.append(ultraup_mean[0])
        ultradown_amp.append(ultradown_mean[0])
        # RMS
        ultra_mean = np.mean([np.sqrt(ultraup_mean**2),np.sqrt(ultradown_mean**2)])
        ultra_amp.append(ultra_mean)
    
    selectup_gen_evoked.append(upgen_select_dict)
    selectdown_gen_evoked.append(downgen_select_dict) 
    selectup_tail_evoked.append(uptail_select_dict)
    selectdown_tail_evoked.append(downtail_select_dict) 
    selectup_ultra_evoked.append(uptail_select_dict)
    selectdown_ultra_evoked.append(downtail_select_dict) 

c1_long_df['attention'] = mid_att
c1_long_df['expectation'] = mid_pred
c1_long_df['general_amp'] = general_amp
c1_long_df['general_up_amp'] = genup_amp
c1_long_df['general_down_amp'] = gendown_amp
c1_long_df['tailored_amp'] = tailored_amp
c1_long_df['tailored_up_amp'] = tailup_amp
c1_long_df['tailored_down_amp'] = taildown_amp
c1_long_df['ultra_amp'] = ultra_amp
c1_long_df['ultra_up_amp'] = ultraup_amp
c1_long_df['ultra_down_amp'] = ultradown_amp


In [ ]:
c1_long_df.to_csv(r'C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes\c1_df.csv',index= False)

In [13]:
general_up_evokeds = {}
general_down_evokeds = {}

tailor_up_evokeds = {}
tailor_down_evokeds = {}

ultra_up_evokeds = {}
ultra_down_evokeds = {}

for i,cond in enumerate(conditions):
    list_for_upgencond = []
    list_for_downgencond = []

    list_for_uptailcond = []
    list_for_downtailcond = []

    list_for_upultracond = []
    list_for_downultracond = []

    for ind,sub in enumerate(sub_list):
        list_for_upgencond.append(selectup_gen_evoked[i][cond])
        list_for_downgencond.append(selectdown_gen_evoked[i][cond])

        list_for_uptailcond.append(selectup_tail_evoked[i][cond])
        list_for_downtailcond.append(selectdown_tail_evoked[i][cond])

        list_for_upultracond.append(selectup_ultra_evoked[i][cond])
        list_for_downultracond.append(selectdown_ultra_evoked[i][cond])


In [18]:

evoked_ga_up = [mne.grand_average(evokeds_up['reg_att']),
               mne.grand_average(evokeds_up['odd_att']),
               mne.grand_average(evokeds_up['reg_unatt']),
               mne.grand_average(evokeds_up['odd_unatt'])]

evoked_ga_down = [mne.grand_average(evokeds_down['reg_att']),
               mne.grand_average(evokeds_down['odd_att']),
               mne.grand_average(evokeds_down['reg_unatt']),
               mne.grand_average(evokeds_down['odd_unatt'])]

evoked_ga = []

for i,(up,down) in enumerate(zip(evoked_ga_up,evoked_ga_down)):
    rms_data = np.mean([np.sqrt(up.data**2),np.sqrt(down.data**2)],axis=0)
    rms_evoked = mne.EvokedArray(rms_data,info=up.info, tmin=up.times[0])
    evoked_ga.append(rms_evoked)




conditions = ('attended reg','attended odd','unattended reg','unattended odd')



Identifying common channels ...


Identifying common channels ...
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...


In [26]:

norm = dict(zip(conditions,evoked_ga))
# norm = general_down_evokeds
epoch_set1 = norm
scale = [-6, 6]
# mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Pz', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1,picks=['POz'],vlines=[0.05,0.1],ylim=dict(eeg=scale))
# mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Oz', vlines=[0.05,0.1],ylim=dict(eeg=scale))

[<Figure size 800x600 with 2 Axes>]

In [ ]:
tailor_up_evokeds['reg_att'].plot(ylim=dict(eeg=scale))
tailor_up_evokeds['odd_att'].plot(ylim=dict(eeg=scale))
tailor_up_evokeds['reg_unatt'].plot(ylim=dict(eeg=scale))
tailor_up_evokeds['odd_unatt'].plot(ylim=dict(eeg=scale))

In [ ]:
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'C1', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'C2', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Cz', vlines=[0.05,0.1],ylim=dict(eeg=scale))

mne.viz.plot_compare_evokeds(epoch_set1, picks= 'P1', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'P2', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Pz', vlines=[0.05,0.1],ylim=dict(eeg=scale))

mne.viz.plot_compare_evokeds(epoch_set1, picks= 'PO3', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'PO4', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'POz', vlines=[0.05,0.1],ylim=dict(eeg=scale))

mne.viz.plot_compare_evokeds(epoch_set1, picks= 'O1', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'O2', vlines=[0.05,0.1],ylim=dict(eeg=scale))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Oz', vlines=[0.05,0.1],ylim=dict(eeg=scale))

In [ ]:
"""

"""
times = np.linspace(0.1, 0.5, 7)
vlim=(-3, 3)

evoked_pred[0].plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
evoked_pred[1].plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
evoked_pred[2].plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
evoked_pred[3].plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)

In [ ]:
times = np.linspace(0.05, 0.11, 5)
group4_pos1_unatt.plot_topomap(ch_type="eeg", times=times, colorbar=True)
group4_pos3_att.plot_topomap(ch_type="eeg", times=times, colorbar=True)
group4_pos1_unatt.plot_topomap(ch_type="eeg", times=times, colorbar=True)
group4_pos3_att.plot_topomap(ch_type="eeg", times=times, colorbar=True)

### Time course of c1 peaks and means

In [ ]:
""" 
Time course first sequence position
"""
# attended
ep_df_att_1 = ep_att_1.metadata
ep_df_att_1['poz_c1_mean'] = ep_att_1.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_att_2 = ep_att_2.metadata
ep_df_att_2['poz_c1_mean'] = ep_att_2.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_att_3 = ep_att_3.metadata
ep_df_att_3['poz_c1_mean'] = ep_att_3.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_att_4 = ep_att_4.metadata
ep_df_att_4['poz_c1_mean'] = ep_att_4.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
# unattend
ep_df_unatt_1= ep_unatt_1.metadata
ep_df_unatt_1['poz_c1_mean'] = ep_unatt_1.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_unatt_2 = ep_unatt_2.metadata
ep_df_unatt_2['poz_c1_mean'] = ep_unatt_2.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_unatt_3 = ep_unatt_3.metadata
ep_df_unatt_3['poz_c1_mean'] = ep_unatt_3.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_unatt_4 = ep_unatt_4.metadata
ep_df_unatt_4['poz_c1_mean'] = ep_unatt_4.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)


In [ ]:
""" 
Time course
"""
position = ['pos1','pos2','pos3','pos4']
condition = 'attention == "attended" & expected == "regular" & start_position == 2'
pos1_win_data = c1_window_epochs[condition][position[0]].get_data(picks='POz')
pos2_win_data = c1_window_epochs[condition][position[1]].get_data(picks='POz')
pos3_win_data = c1_window_epochs[condition][position[2]].get_data(picks='POz')
pos4_win_data = c1_window_epochs[condition][position[3]].get_data(picks='POz')

In [ ]:
mean_c1_1 = pos1_win_data.mean(axis=2).squeeze()
mean_c1_2 = pos2_win_data.mean(axis=2).squeeze()
mean_c1_3 = pos3_win_data.mean(axis=2).squeeze()
mean_c1_4 = pos4_win_data.mean(axis=2).squeeze()

max_c1_1 = pos1_win_data.max(axis=2).squeeze()
max_c1_2 = pos2_win_data.max(axis=2).squeeze()
max_c1_3 = pos3_win_data.max(axis=2).squeeze()
max_c1_4 = pos4_win_data.max(axis=2).squeeze()

mean_c1_list = [mean_c1_1,mean_c1_2,mean_c1_3,mean_c1_4]
max_c1_list = [max_c1_1,max_c1_2,max_c1_3,max_c1_4]

In [ ]:
"""
Plotting mean c1 voltage time wise
"""
plt.rcParams["figure.figsize"] = [18,10]
fig, axs = plt.subplots(4,2, sharey=True)

sns.regplot(data = ep_df_att_1, x = 'trial', y = 'poz_c1_mean', ax = axs[0,0],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_1, x = 'trial', y = 'poz_c1_mean', ax = axs[1,0],scatter_kws={'s': 10, 'color': 'green'})
sns.regplot(data = ep_df_att_2, x = 'trial', y = 'poz_c1_mean', ax = axs[0,1],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_2, x = 'trial', y = 'poz_c1_mean', ax = axs[1,1],scatter_kws={'s': 10, 'color': 'green'})
sns.regplot(data = ep_df_att_3, x = 'trial', y = 'poz_c1_mean', ax = axs[2,1],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_3, x = 'trial', y = 'poz_c1_mean', ax = axs[3,1],scatter_kws={'s': 10, 'color': 'green'})
sns.regplot(data = ep_df_att_4, x = 'trial', y = 'poz_c1_mean', ax = axs[2,0],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_4, x = 'trial', y = 'poz_c1_mean', ax = axs[3,0],scatter_kws={'s': 10, 'color': 'green'})
plt.tight_layout()
plt.show()
# sns.regplot(x = np.arange(len(mean_c1_list[1])), y = mean_c1_list[1], ax=axs[0,1])
# sns.regplot(x = np.arange(len(mean_c1_list[2])), y = mean_c1_list[2], ax=axs[1,1])
# sns.regplot(x = np.arange(len(mean_c1_list[3])), y = mean_c1_list[3], ax=axs[1,0])



In [ ]:
"""
Plotting max c1 voltage time wise
"""
plt.rcParams["figure.figsize"] = [18,10]
fig, axs = plt.subplots(2,2, sharey=True)

sns.regplot(x = np.arange(len(max_c1_list[0])), y = max_c1_list[0], ax=axs[0,0])
sns.regplot(x = np.arange(len(max_c1_list[1])), y = max_c1_list[1], ax=axs[0,1])
sns.regplot(x = np.arange(len(max_c1_list[2])), y = max_c1_list[2], ax=axs[1,1])
sns.regplot(x = np.arange(len(max_c1_list[3])), y = max_c1_list[3], ax=axs[1,0])

plt.suptitle('max values '+ condition)
plt.tight_layout()
plt.show()

### Time frequency exploration

In [ ]:
spec_first_att = epochs_attended['pos4/seq2'].average().compute_psd()
spec_last_att = epochs_attended['seq3'].average().compute_psd()
spec_first_unatt = epochs_unattended['seq1'].average().compute_psd()
spec_last_unatt = epochs_unattended['seq3'].average().compute_psd()

In [ ]:
spec_first_att.plot_topomap(ch_type="eeg")
spec_first_unatt.plot_topomap(ch_type="eeg")

In [ ]:
freqs = np.logspace(*np.log10([8, 13]), num=8)
n_cycles = freqs / 3.0  # different number of cycle per frequency
power, itc = tfr_morlet(
    epochs_attended['pos4/seq2'],
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    n_jobs=None,
)

In [ ]:

power.plot([27], baseline=(-0.09, 0), mode="logratio", title=power.ch_names[27])

fig, axes = plt.subplots(1, 2, figsize=(7, 4), layout="constrained")
topomap_kw = dict(
    ch_type="eeg", tmin=0.09, tmax=0.5, baseline=(-0.09, 0), mode="logratio", show=False
)
plot_dict = dict(Alpha=dict(fmin=8, fmax=13))
for ax, (title, fmin_fmax) in zip(axes, plot_dict.items()):
    power.plot_topomap(**fmin_fmax, axes=ax, **topomap_kw)
    ax.set_title(title)